# Detecting EMA cross traps by using NEAT

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import os

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import neat

### Load Price Data

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(csv_file)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

/home/zuongthao/PycharmProjects/algo-stock/ai-chungkhoan/VN30F1M/NEAT_for_EMA_cross/VN30F1M_5minutes.csv
remote


In [4]:
data = dataset.copy()

In [5]:
data = data[data.index > '2020-11-01 00:00:00']

In [6]:
data["ema_fast"] = ta.ema(data["Close"], length=20)
data["ema_low"] = ta.ema(data["Close"], length=250)
data["ema_cross"] = ((data["ema_fast"] > data["ema_low"]) & (data["ema_fast"].shift(1) <= data["ema_low"].shift(1)) | (data["ema_fast"] < data["ema_low"]) & (data["ema_fast"].shift(1) >= data["ema_low"].shift(1)))

## Calculate some common features

In [7]:
data["ATR"] = ta.atr(data["High"], data["Low"], data["Close"], length=14)  # Volatility
data["RSI"] = ta.rsi(data["Close"], length=14)  # Momentum indicator

## TRAP labeling

In [8]:
def is_trap(r):
    trap = ''
    if r['ema_cross'] == True:
        if r['ema_fast'] > r['ema_low']:
            # Cross up
            if r['min_low_1dlater'] < r['Close'] - 3.5:
                trap = 1
            else:
                trap = 0
        else:
            # Cross down
            if r['max_high_1dlater'] > r['Close'] + 3.5:
                trap = 1
            else:
                trap = 0
    return trap

In [9]:
data['max_high_1dlater'] = data['High'].shift(-51).rolling(51).max()
data['min_low_1dlater'] = data['Low'].shift(-51).rolling(51).min()
data['trap'] = data.apply(lambda r: is_trap(r), axis=1)

In [10]:
# cross_data = data[data.ema_cross == True]
# len(cross_data[cross_data.trap == 0]) / len(cross_data['trap'])

## Features

In [11]:
data = data[(100 * data.index.hour + data.index.minute != 1130) & (100 * data.index.hour + data.index.minute != 1430)]
data['pct_change'] = data['Volume'].pct_change()
data['pct_change_s1'] = data['pct_change'].shift(1)
data['pct_change_s2'] = data['pct_change'].shift(2)
data['pct_change_s3'] = data['pct_change'].shift(3)
data['pct_change_s4'] = data['pct_change'].shift(4)
data['pct_change_s5'] = data['pct_change'].shift(5)
data['pct_change_s6'] = data['pct_change'].shift(6)
data['pct_change_s7'] = data['pct_change'].shift(7)
data['pct_change_s8'] = data['pct_change'].shift(8)
cross_up_data = data[(data.ema_cross == True) & (data.ema_fast > data.ema_low)]
cross_up_data.dropna(inplace=True)

In [12]:
len(cross_up_data)

174

In [13]:
X = cross_up_data[['pct_change', 'pct_change_s1', 'pct_change_s2', 'pct_change_s3', 'pct_change_s4', 'pct_change_s5', 'pct_change_s6', 'pct_change_s7', 'pct_change_s8', "trap"]]

# Train-Test Split
X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)

In [14]:
len(X_train)

130

In [15]:
len(X_test)

44

In [16]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for move_index, row in X_train.iterrows():
            inputs = [row['pct_change'], row['pct_change_s1'], row['pct_change_s2'], row['pct_change_s3'], row['pct_change_s4'], row["pct_change_s5"], row["pct_change_s6"], row["pct_change_s7"], row["pct_change_s8"]]
            expected_output = row['trap']
            output = net.activate(inputs)
            genome.fitness -= (output[0] - expected_output) ** 2


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    # p.add_reporter(neat.StdOutReporter(True))
    # stats = neat.StatisticsReporter()
    # p.add_reporter(stats)

    # Run for up to 100 generations.
    winner = p.run(eval_genomes, 100)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    return neat.nn.FeedForwardNetwork.create(winner, config)

In [17]:
%%time
config_path = os.path.join(current_dir, 'style-mix-1.cfg')
best_brain = run(config_path)


Best genome:
Key: 13785
Fitness: -22.965104879094884
Nodes:
	0 DefaultNodeGene(key=0, bias=0.17824255235263908, response=1.0, activation=sigmoid, aggregation=sum)
	500 DefaultNodeGene(key=500, bias=0.36624503184489576, response=1.0, activation=sigmoid, aggregation=sum)
	588 DefaultNodeGene(key=588, bias=0.9538200260719973, response=1.0, activation=sigmoid, aggregation=sum)
	821 DefaultNodeGene(key=821, bias=-0.08603640942159979, response=1.0, activation=sigmoid, aggregation=sum)
	1393 DefaultNodeGene(key=1393, bias=-1.2426702762362283, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-6, 0), weight=-0.15229006146050328, enabled=True)
	DefaultConnectionGene(key=(-5, 0), weight=-2.0386127066242627, enabled=False)
	DefaultConnectionGene(key=(-4, 0), weight=-1.1829724601461262, enabled=False)
	DefaultConnectionGene(key=(-4, 500), weight=-1.1357680314234542, enabled=False)
	DefaultConnectionGene(key=(-4, 821), weight=0.4169817412361126, enabled=Tr

In [18]:
best_brain

In [19]:
# Show output of the most fit genome against training data.
outputs = []
for i, row in X_test.iterrows():
    inputs = [row['pct_change'], row['pct_change_s1'], row['pct_change_s2'], row['pct_change_s3'], row['pct_change_s4'], row["pct_change_s5"], row["pct_change_s6"], row["pct_change_s7"], row["pct_change_s8"]]
    expected_output = row['trap']
    output = best_brain.activate(inputs)
    outputs.append(round(output[0]))
    # print("input {!r}, expected output {!r}, got {!r}".format(inputs, expected_output, output))

In [20]:
expected_outputs = X_test['trap'].to_list()
# Evaluate Performance
print("Accuracy:", accuracy_score(expected_outputs, outputs))

Accuracy: 0.6590909090909091


In [21]:
X_test

,pct_change,pct_change_s1,pct_change_s2,pct_change_s3,pct_change_s4,pct_change_s5,pct_change_s6,pct_change_s7,pct_change_s8,trap
Date,,,,,,,,,,
2024-07-19 10:00:00,-0.168502,-0.192530,1.928385,-0.183413,-0.472222,-0.585340,-0.027935,1.088826,0.361531,1
2024-04-10 09:25:00,-0.364326,-0.270342,-0.193420,0.454436,-0.234352,-0.165976,-0.039522,0.050178,-0.150618,1
2022-12-19 10:55:00,-0.465552,0.010456,0.559147,1.598479,-0.228707,-0.125519,0.041455,-0.242097,-0.166988,1
2023-10-17 11:25:00,-0.259201,0.869322,-0.429912,0.470475,-0.036170,0.238880,-0.142655,-0.548872,-0.174782,1
2024-02-27 10:20:00,0.167513,1.162459,-0.316579,-0.747538,0.324802,0.913806,1.428571,-0.195214,0.160675,1
2024-01-30 14:25:00,0.365319,1.214950,0.323154,-0.105585,0.036771,-0.468553,0.084267,3.372737,-0.401530,1
2021-12-02 09:35:00,0.092286,-0.015614,-0.460153,0.600245,0.000000,-0.169129,-0.784191,0.746544,-0.366885,1
2021-05-18 13:50:00,1.083577,-0.163948,0.333092,-0.388248,0.070242,-0.160558,-0.188884,0.548799,-0.032446,1
2023-09-20 14:25:00,0.262827,-0.041405,-0.241169,1.994867,-0.245969,-0.354117,1.882706,-0.509908,0.314676,1
